In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import os
from modules.env import MazeEnvironment
import seaborn as sns
from modules.base import SRMB
from modules.model import TAGTransfer
import matplotlib
import numpy as np
from collections import deque
from scipy.stats import ttest_rel

from tests.guo_2024 import sparsify_transition, split_reconstruct_SR
from matplotlib import cm
from sklearn.manifold import Isomap
from scipy.ndimage import gaussian_filter
from tests.nakai_2024 import transform_corner, fuzz_features
from tests.guo_2024 import sparsify_transition, split_reconstruct_SR

%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['ps.fonttype'] = 42
rcParams['font.sans-serif'] = ['Arial']  # 예시: Arial로 설정
rcParams['font.family'] = 'sans-serif'

sns.set_style(
    'ticks',
    rc={
        'xtick.direction': 'in',  # x축 tick을 안쪽으로
        'ytick.direction': 'in',  # y축 tick을 안쪽으로
    }
)
rcParams["figure.dpi"] = 600
# Set the font to Arial
big_font = 20
mid_font = 15
small_font = 12

indicator_font = big_font
legend_font = mid_font
title_font = mid_font
axis_font = mid_font
tick_font = small_font



subplot_scale_factor = 3

indicator_x_loc = -0.1
indicator_y_loc = 1.15

node_size = 300
edge_width = 4


In [2]:
junction_color = "#416788"
deadend_color = "#646464"
edge_color = "#E1DAD2"

palette = ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600']

In [3]:
out_dir = 'data/neural_preds/guo_2024'
task_type = 0
s1 = 0.2
s2 = 0.1
threshold = 230

num_row, num_column = 9, 9
scale_factor = 5

env = MazeEnvironment(num_row, num_column)
blocks = []
for r in range(1, 8):
    for c in range(1, 8):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks
nonblocks = env.nonblocks

gamma = 0.995
n_neighbors = 13

sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
fraction = s2
sr.transition_matrix = sparsify_transition(sr.transition_matrix, zero_fraction=fraction, seed=43)
sr.compute_sr()
eigvec, eigval = sr.decompose_map_eigen()
SR_before, SR_after = split_reconstruct_SR(eigvec.T, eigval, threshold=threshold)
embs = []
for SR in (sr.sr, SR_before, SR_after):
    emb = Isomap(
        n_components=2, n_neighbors=n_neighbors, eigen_solver="dense"
    ).fit_transform(SR[nonblocks][:, nonblocks])
    embs.append(emb)
emb_original_s2, emb_before_s2, emb_after_s2 = embs[0], embs[1], embs[2]


fraction = s1
sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
sr.transition_matrix = sparsify_transition(sr.transition_matrix, zero_fraction=fraction, seed=43)
sr.compute_sr()

eigvec, eigval = sr.decompose_map_eigen()
SR_before, SR_after = split_reconstruct_SR(eigvec.T, eigval, threshold=threshold)
embs = []
for SR in (sr.sr, SR_before, SR_after):
    emb = Isomap(
        n_components=2, n_neighbors=n_neighbors, eigen_solver="dense"
    ).fit_transform(SR[nonblocks][:, nonblocks])
    embs.append(emb)
emb_original_s1, emb_before_s1, emb_after_s1 = embs[0], embs[1], embs[2]
task_type = 0
s1 = 0.2
s2 = 0.1
threshold = 230

num_row, num_column = 9, 9
# for scale_factor in range(1, 6):
scale_factor = 5

env = MazeEnvironment(num_row, num_column)
blocks = []
for r in range(1, 8):
    for c in range(1, 8):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks
nonblocks = env.nonblocks

gamma = 0.995
n_neighbors = 13

sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
fraction = s2
sr.transition_matrix = sparsify_transition(sr.transition_matrix, zero_fraction=fraction, seed=43)
sr.compute_sr()
eigvec, eigval = sr.decompose_map_eigen()
SR_before, SR_after = split_reconstruct_SR(eigvec.T, eigval, threshold=threshold)
embs = []
for SR in (sr.sr, SR_before, SR_after):
    emb = Isomap(
        n_components=2, n_neighbors=n_neighbors, eigen_solver="dense"
    ).fit_transform(SR[nonblocks][:, nonblocks])
    embs.append(emb)
emb_original_s2, emb_before_s2, emb_after_s2 = embs[0], embs[1], embs[2]


fraction = s1
sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
sr.transition_matrix = sparsify_transition(sr.transition_matrix, zero_fraction=fraction, seed=43)
sr.compute_sr()

eigvec, eigval = sr.decompose_map_eigen()
SR_before, SR_after = split_reconstruct_SR(eigvec.T, eigval, threshold=threshold)
embs = []
for SR in (sr.sr, SR_before, SR_after):
    emb = Isomap(
        n_components=2, n_neighbors=n_neighbors, eigen_solver="dense"
    ).fit_transform(SR[nonblocks][:, nonblocks])
    embs.append(emb)
emb_original_s1, emb_before_s1, emb_after_s1 = embs[0], embs[1], embs[2]


num_row, num_column = 6, 3
fig, axs = plt.subplots(num_row, num_column, figsize=(num_column * 3, num_row * 3), constrained_layout=True)
for ax in axs.flat:
    ax.set_box_aspect(1)

for ax in axs.flat:
    if ax is not axs[4, 0]:
        ax.set_xticks([])
        ax.set_yticks([])

axs[1, 0].remove()
axs[1, 1].remove()
axs[1, 0] = fig.add_subplot(num_row, num_column, 1*num_column + 1, projection='3d')
axs[1, 1] = fig.add_subplot(num_row, num_column, 1*num_column + 2, projection='3d')

num_row, num_column = 9, 9
# for scale_factor in range(1, 6):
scale_factor = 5

env = MazeEnvironment(num_row, num_column)
blocks = []
for r in range(1, 8):
    for c in range(1, 4):
        blocks.append(env.index_to_state(r, c))
    for c in range(5, 8):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks

tag = TAGTransfer(num_row * scale_factor, num_column * scale_factor)
srmb = SRMB(num_row * scale_factor, num_column * scale_factor)

srmb.update_map_structure(blocks)
tag.set_map(srmb.sr)
tag.update_grid()

tag_with_border_result = np.load("data/neural_preds/barry_2007/tag_with_border_result.npy")
tag_without_border_result = np.load("data/neural_preds/barry_2007/tag_without_border_result.npy")


def plot_rescaling(data, title, ax):
    # compute delta from baseline=1.0
    delta = data - 1.0
    # split into horiz. / vert.
    horiz = delta[0::2]
    vert  = delta[1::2]

    dims = np.arange(len(horiz))  # [0,1,2]
    width = 0.35
    
    # positions for   each bar: for each dim, horiz at dim-width/2, vert at dim+width/2
    tick_positions = []
    tick_labels = []
    for _ in dims:
        tick_positions.extend([_ - width/2, _ + width/2])
        tick_labels.extend(['Horiz.', 'Vert.'])

    # bars start at y=1.0, height = (value − 1.0)
    ax.bar(dims - width/2, horiz, width,
           bottom=1.0,
           label='Horiz.', color='#344195')
    ax.bar(dims + width/2, vert, width,
           bottom=1.0,
           label='Vert.',  color='#e82d30')

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_ylim(0.6, 1.5)
    ax.axhline(1.0, color='#989999')             # baseline
    ax.axhline(0.7, color='#989999', linestyle='--')
    ax.set_xlabel('Dimension')
    ax.set_ylabel('Rescaling\n(relative to baseline)')
    ax.set_title(title)
    return ax

# generate both figures
plot_rescaling(tag_with_border_result,    'With Border', axs[0, 1])
plot_rescaling(tag_without_border_result, 'Without Border', axs[0, 2])


def multi_source_bfs(sources, n_states, graph):
    """
    다중 소스 BFS로 각 상태까지의 geodesic distance를 계산합니다.
    sources: 거리 계산의 시작점 리스트
    """
    dist = np.full(n_states, np.inf)
    q = deque()
    for s in sources:
        dist[s] = 0
        q.append(s)
    while q:
        u = q.popleft()
        for v in graph.get(u, []):
            if dist[v] > dist[u] + 1:
                dist[v] = dist[u] + 1
                q.append(v)
    return dist


scale = 18

corner, border = tag.structure_head_corner_stack(False)
corner = transform_corner(corner)
corner = fuzz_features(
    corner, num_row * scale_factor, num_column * scale_factor, env.blocks, scale=scale
)

dim_sr_list = np.load('data/neural_preds/nakai_2024/dim_sr.npy')
dim_border_corner_list = np.load('data/neural_preds/nakai_2024/dim_border_corner.npy')


def plot_best_performance_comparison_single_border_simple(ax, border_max_shift, data_dir, sr_param_values):
    """Plot bar chart comparing dimensionality at best performance parameters for a specific border_max_shift"""
    # Load all data automatically
    sr_dim_means = np.load(os.path.join(data_dir, "sr_dim_means_processed.npy"))
    sr_dim_stds = np.load(os.path.join(data_dir, "sr_dim_stds_processed.npy"))
    bc_dim_means = np.load(os.path.join(data_dir, "bc_dim_means_processed.npy"))
    bc_dim_stds = np.load(os.path.join(data_dir, "bc_dim_stds_processed.npy"))
    sr_means = np.load(os.path.join(data_dir, "sr_means_processed.npy"))
    sr_stds = np.load(os.path.join(data_dir, "sr_stds_processed.npy"))
    bc_means = np.load(os.path.join(data_dir, "bc_raw_means_processed.npy"))
    bc_stds = np.load(os.path.join(data_dir, "bc_raw_stds_processed.npy"))
    
    sr_optimal_idx = np.argmax(sr_means)
    sr_optimal_param = sr_param_values[sr_optimal_idx]
    sr_dim_mean = sr_dim_means[sr_optimal_idx]
    sr_dim_std = sr_dim_stds[sr_optimal_idx]
    
    bc_optimal_idx = np.argmax(bc_means[border_max_shift - 1])
    bc_optimal_param = sr_param_values[sr_optimal_idx]  # Using same parameter mapping as original

    # Get dimensionality values at optimal parameters
    bc_dim_mean = bc_dim_means[border_max_shift - 1][bc_optimal_idx]
    bc_dim_std = bc_dim_stds[border_max_shift - 1][bc_optimal_idx]
        
    # Load raw dimensionality data for statistical testing
    sr_raw_data = []
    bc_raw_data = []
    
    # Load raw dimensionality data for optimal parameters
    for overall_index in range(1, 1001):
        try:
            sr_dim = np.load(os.path.join(data_dir, f"raw_data/sr_dim_{overall_index}.npy"))
            bc_dim = np.load(os.path.join(data_dir, f"raw_data/bc_dim_{overall_index}_{border_max_shift}.npy"))
            
            # Extract dimensionality values for optimal parameters
            if len(sr_dim) > sr_optimal_idx:
                sr_value = sr_dim[sr_optimal_idx]
                if not np.isnan(sr_value):
                    sr_raw_data.append(sr_value)
            
            if len(bc_dim) > bc_optimal_idx:
                bc_value = bc_dim[bc_optimal_idx]
                if not np.isnan(bc_value):
                    bc_raw_data.append(bc_value)
                    
        except (FileNotFoundError, IndexError):
            continue
    
    # Perform t-test if we have enough data
    if len(sr_raw_data) > 1 and len(bc_raw_data) > 1:
        _, p_val = ttest_rel(sr_raw_data, bc_raw_data)
        if isinstance(p_val, tuple):
            p_val = p_val[1]
        if not isinstance(p_val, float):
            p_val = float(np.asarray(p_val))
    else:
        p_val = np.nan  # Not enough data for testing

    # Create bar plot
    ax.bar(['CA1', 'SUB'], [sr_dim_mean, bc_dim_mean], 
                  yerr=[sr_dim_std, bc_dim_std], 
                  capsize=5, color=['#1f77b4', '#ff7f0e'], alpha=0.8)
    
    
    # Add significance indicator
    y_max = max(sr_dim_mean + sr_dim_std, bc_dim_mean + bc_dim_std) + 0.05
    bar_height = y_max * 0.1
    ax.plot([0, 0, 1, 1], [y_max, y_max + bar_height, y_max + bar_height, y_max], "k-", linewidth=1)
    
    # Determine significance level
    if p_val < 0.0001:
        asterisk = "****"
    elif p_val < 0.001:
        asterisk = "***"
    elif p_val < 0.01:
        asterisk = "**"
    elif p_val < 0.05:
        asterisk = "*"
    else:
        asterisk = "ns"
    
    ax.text(0.5, y_max + bar_height, asterisk, ha="center", va="bottom", fontsize=8)
    
    ax.set_ylabel('Dimensionality', fontsize=8)
    ax.set_title(f'Border Max Shift = {border_max_shift}', fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='both', which='major', labelsize=7)

data_dir = "data/neural_preds/nakai_2024"
gamma_values = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99, 0.995, 0.999])


# Call the simplified function for border_max_shift = 16
plot_best_performance_comparison_single_border_simple(
    ax=axs[1, 2],
    border_max_shift=16,
    data_dir=data_dir,
    sr_param_values=gamma_values
)

num_row = 9
num_column = 9
scale_factor = 5

scale =  - 1 / np.log(0.99)
# — 1) feature 생성 (생략) —
border_max_shift = 16
corner, border = tag.structure_head_corner_stack(False)
corner = transform_corner(corner)
corner = fuzz_features(corner, num_row*scale_factor, num_column*scale_factor, env.blocks, scale=scale)
border = fuzz_features(border, num_row*scale_factor, num_column*scale_factor, env.blocks, scale=scale)
border_corner = np.concatenate((border, corner), axis=1)

# grid 크기
H = num_row * scale_factor
W = num_column * scale_factor
center_row, center_col = H//2, W//2

def compute_t_for_left(r, c):
    if c == center_col and 0 <= r <= center_row*2:
        return (center_row*2 - r) / (center_row*2) * 0.33

    if r == 0 and 0 <= c <= center_col:
        return 0.33 + (center_col - c) / center_col * (0.50 - 0.33)

    if c == 0 and 0 <= r <= center_row*2:
        return 0.50 + (r / (center_row*2)) * (0.75 - 0.50)

    if r == center_row*2 and 0 <= c <= center_col:
        return 0.75 + (c / center_col) * (1.00 - 0.75)

    pts = [
        ((center_row*2, center_col), 0.00),  
        ((0, center_col), 0.33),             
        ((0, 0), 0.50),                      
        ((center_row*2, 0), 0.75)            
    ]
    dists = [abs(r - pr) + abs(c - pc) for (pr, pc), _ in pts]
    idx_min = int(np.argmin(dists))
    return pts[idx_min][1]

def compute_t_for_right(r, c):
    if c == center_col and 0 <= r <= center_row*2:
        return (center_row*2 - r) / (center_row*2) * 0.33

    if r == 0 and center_col <= c <= center_col*2:
        return 0.33 + (c - center_col) / center_col * (0.50 - 0.33)

    if c == center_col*2 and 0 <= r <= center_row*2:
        return 0.50 + (r / (center_row*2)) * (0.75 - 0.50)

    if r == center_row*2 and center_col <= c <= center_col*2:
        return 0.75 + ((center_col*2 - c) / center_col) * (1.00 - 0.75)

    pts = [
        ((center_row*2, center_col), 0.00),    
        ((0, center_col), 0.33),               
        ((0, center_col*2), 0.50),             
        ((center_row*2, center_col*2), 0.75)   
    ]
    dists = [abs(r - pr) + abs(c - pc) for (pr, pc), _ in pts]
    idx_min = int(np.argmin(dists))
    return pts[idx_min][1]

t_map = np.zeros((H, W), dtype=float)
for r in range(H):
    for c in range(W):
        if c <= center_col:
            t_map[r, c] = compute_t_for_left(r, c)
        else:
            t_map[r, c] = compute_t_for_right(r, c)

# sigma는 그라데이션 부드러움 정도 (1~3 사이로 조정)
t_map_smooth = gaussian_filter(t_map, sigma=3)

# — 4) nonblocks 점들 좌표, 컬러 샘플링 —
nonb = np.array(env.nonblocks, dtype=int)
rows = nonb // W
cols = nonb % W

colors_rgba = np.zeros((len(nonb), 4), dtype=float)
for i, (r, c) in enumerate(zip(rows, cols)):
    if c <= center_col:
        colors_rgba[i] = cm.spring(t_map_smooth[r, c])
    else:
        colors_rgba[i] = cm.winter(t_map_smooth[r, c])



n_neighbors = 20
isomap = Isomap(n_components=3, n_neighbors=n_neighbors)

pts3d = isomap.fit_transform(srmb.sr[nonb])

axs[1, 0].scatter(
    pts3d[:, 0],  # 필요에 따라 축 반전
    pts3d[:, 1]*-1,
    pts3d[:, 2],
    c=colors_rgba,
    s=10,
    depthshade=True
)
axs[1, 0].view_init(elev=60, azim=-100)
# 2) 축과 격자 모두 끄기
axs[1, 0].set_axis_off()
axs[1, 0].grid(False)


isomap = Isomap(n_components=3, n_neighbors=n_neighbors)
pts3d = isomap.fit_transform(border_corner[nonb])

axs[1, 1].scatter(
    pts3d[:, 0] * -1,  # 필요에 따라 축 반전
    pts3d[:, 1],
    pts3d[:, 2],
    c=colors_rgba,
    s=10,
    depthshade=True
)
axs[1, 1].view_init(elev=60, azim=-10)
# 2) 축과 격자 모두 끄기
axs[1, 1].set_axis_off()
axs[1, 1].grid(False)


# Create custom legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#808080', edgecolor='black', label='With corner'),
                   Patch(facecolor='#C0C0C0', edgecolor='black', label='Without corner')]

gamma = 0.7  
n_neighbors = 13  


def get_color_t(x, y):
    colors_corners = np.array([
        [129, 39, 216],   
        [83, 204, 93],   
        [218, 91, 45],   
        [216, 216, 216] 
    ])
    colors_corners = colors_corners / 255
    w_bl = (1-x)*(1-y)  
    w_tl = (1-x)*y      
    w_r = x
    w_center = np.exp(-((x-0.2)**2 + (y-0.5)**2) / 0.02)  
    weights = np.array([w_tl, w_bl, w_r, w_center])
    weights /= weights.sum()
    color = np.dot(weights, colors_corners)
    return color

def get_color_o(x, y):
    colors_corners = np.array([
        [70,223,114],   
        [34,28,219],   
        [221, 221, 21],  
        [216,15,118],  
    ])
    colors_corners = colors_corners / 255
    w_bl = (1-x)*(1-y)  
    w_tl = (1-x)*y      
    w_br = x*(1-y)      
    w_tr = x*y          
    weights = np.array([w_bl, w_tl, w_br, w_tr])
    weights /= weights.sum()
    color = np.dot(weights, colors_corners)
    return color

def get_color_h(x, y):
    colors_corners = np.array([
        [70,223,114],  
        [34,28,219], 
        [221, 221, 21],
        [216,15,118],  
        [216, 216, 216] 
    ])
    colors_corners = colors_corners / 255

    w_bl = (1-x)*(1-y) 
    w_tl = (1-x)*y      
    w_br = x*(1-y)     
    w_tr = x*y         
    w_center = np.exp(-((x-0.5)**2 + (y-0.5)**2) / 0.02)  
    weights = np.array([w_bl, w_tl, w_br, w_tr, w_center])
    weights /= weights.sum()
    color = np.dot(weights, colors_corners)
    return color


num_row, num_column = 9, 9
scale_factor = 5
env = MazeEnvironment(num_row, num_column)
nonblocks = []
for r in range(9):
    nonblocks.append(env.index_to_state(r, 0))
for c in range(9):
    nonblocks.append(env.index_to_state(4, c))
nonblocks = list(set(nonblocks))
blocks = np.setdiff1d(np.arange(num_row * num_column), nonblocks)
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks

sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
isomap = Isomap(n_components=2, n_neighbors=n_neighbors, eigen_solver="dense")
emb = isomap.fit_transform(sr.sr[env.nonblocks][:, env.nonblocks])

x = emb[:, 0]
y = emb[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())

colors_list = np.array([get_color_t(xi, yi) for xi, yi in zip(x_norm, y_norm)])

ax = axs[2, 0]
ax.scatter(emb[:, 0], emb[:, 1], s=5, c=colors_list)

num_row, num_column = 9, 9

scale_factor=5
env = MazeEnvironment(num_row, num_column)
blocks = []
for r in range(1, 8):
    for c in range(1, 8):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks
sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)

isomap = Isomap(n_components=2, n_neighbors=n_neighbors, eigen_solver="dense")
emb = isomap.fit_transform(sr.sr[env.nonblocks][:, env.nonblocks])

x = emb[:, 0]
y = emb[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())

colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])

ax = axs[2, 1]
ax.scatter(emb[:, 0], emb[:, 1], s=5, c=colors_list)


num_row, num_column = 9, 9
scale_factor = 5
env = MazeEnvironment(num_row, num_column)
blocks = []
for c in range(1, 8):
    for r in range(0, 4):
        blocks.append(env.index_to_state(r, c))
    for r in range(5, 9):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks



sr = SRMB(num_row * scale_factor, num_column * scale_factor, gamma=gamma)
sr.update_map_structure(blocks)
isomap = Isomap(n_components=2, n_neighbors=n_neighbors, eigen_solver="dense")
emb = isomap.fit_transform(sr.sr[env.nonblocks][:, env.nonblocks])


x = emb[:, 0]
y = emb[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())

colors_list = np.array([get_color_h(xi, yi) for xi, yi in zip(x_norm, y_norm)])

ax = axs[2, 2]
ax.scatter(emb[:, 0], emb[:, 1], s=5, c=colors_list)


x = emb_original_s1[:, 0]
y = emb_original_s1[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[3, 0].scatter(emb_original_s1[:, 0], emb_original_s1[:, 1], s=5, c=colors_list)

x = emb_before_s1[:, 0]
y = emb_before_s1[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[3, 1].scatter(emb_before_s1[:, 0], emb_before_s1[:, 1], s=5, c=colors_list)

x = emb_after_s1[:, 0]
y = emb_after_s1[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[3, 2].scatter(emb_after_s1[:, 0], emb_after_s1[:, 1], s=5, c=colors_list)

x = emb_original_s2[:, 0]
y = emb_original_s2[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[4, 0].scatter(emb_original_s2[:, 0], emb_original_s2[:, 1], s=5, c=colors_list)

x = emb_before_s2[:, 0]
y = emb_before_s2[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[4, 1].scatter(emb_before_s2[:, 0], emb_before_s2[:, 1], s=5, c=colors_list)

x = emb_after_s2[:, 0]
y = emb_after_s2[:, 1]
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())
colors_list = np.array([get_color_o(xi, yi) for xi, yi in zip(x_norm, y_norm)])
axs[4, 2].scatter(emb_after_s2[:, 0], emb_after_s2[:, 1], s=5, c=colors_list)


def get_significance(p_val):
    if p_val < 0.0001:
        return "****"
    elif p_val < 0.001:
        return "***"
    elif p_val < 0.01:
        return "**"
    elif p_val < 0.05:
        return "*"
    else:
        return "ns"
ss_s1    = np.load(f'{out_dir}/corrs_before_{task_type}_{s1}_{threshold}.npy') * -1
ss_s2    = np.load(f'{out_dir}/corrs_before_{task_type}_{s2}_{threshold}.npy') * -1
ws_s1    = np.load(f'{out_dir}/corrs_after_{task_type}_{s1}_{threshold}.npy') * -1
ws_s2    = np.load(f'{out_dir}/corrs_after_{task_type}_{s2}_{threshold}.npy') * -1
whole_s1 = np.load(f'{out_dir}/corrs_whole_{task_type}_{s1}_{threshold}.npy') * -1
whole_s2 = np.load(f'{out_dir}/corrs_whole_{task_type}_{s2}_{threshold}.npy') * -1

# Calculate means and stds across seeds
mean_whole_s1, mean_whole_s2 = np.mean(whole_s1, axis=1), np.mean(whole_s2, axis=1)
std_whole_s1, std_whole_s2   = np.std(whole_s1, axis=1), np.std(whole_s2, axis=1)
mean_ss_s1,    mean_ss_s2    = np.mean(ss_s1, axis=1), np.mean(ss_s2, axis=1)
std_ss_s1,     std_ss_s2     = np.std(ss_s1, axis=1), np.std(ss_s2, axis=1)
mean_ws_s1,    mean_ws_s2    = np.mean(ws_s1, axis=1), np.mean(ws_s2, axis=1)
std_ws_s1,     std_ws_s2     = np.std(ws_s1, axis=1), np.std(ws_s2, axis=1)

x = np.arange(3)
# Plot individual seed lines
for seed in range(len(mean_whole_s1)):
    axs[5, 0].plot([x[0] - 0.1, x[0] + 0.1], 
                [mean_whole_s1[seed], mean_whole_s2[seed]], 
                color='gray', alpha=0.3, linewidth=1)
    axs[5, 0].plot([x[1] - 0.1, x[1] + 0.1], 
                [mean_ss_s1[seed], mean_ss_s2[seed]], 
                color='gray', alpha=0.3, linewidth=1)
    axs[5, 0].plot([x[2] - 0.1, x[2] + 0.1], 
                [mean_ws_s1[seed], mean_ws_s2[seed]], 
                color='gray', alpha=0.3, linewidth=1)

# Calculate overall means and stds
mean_whole_s1_all, mean_whole_s2_all = np.mean(mean_whole_s1), np.mean(mean_whole_s2)
std_whole_s1_all, std_whole_s2_all   = np.std(mean_whole_s1), np.std(mean_whole_s2)
mean_ss_s1_all,    mean_ss_s2_all    = np.mean(mean_ss_s1), np.mean(mean_ss_s2)
std_ss_s1_all,     std_ss_s2_all     = np.std(mean_ss_s1), np.std(mean_ss_s2)
mean_ws_s1_all,    mean_ws_s2_all    = np.mean(mean_ws_s1), np.mean(mean_ws_s2)
std_ws_s1_all,     std_ws_s2_all     = np.std(mean_ws_s1), np.std(mean_ws_s2)


t_stat_whole, p_val_whole = ttest_rel(mean_whole_s1, mean_whole_s2)
t_stat_ss, p_val_ss = ttest_rel(mean_ss_s1, mean_ss_s2)
t_stat_ws, p_val_ws = ttest_rel(mean_ws_s1, mean_ws_s2)

# Plot mean lines
axs[5, 0].plot(
    [x[0] - 0.1, x[0] + 0.1], [mean_whole_s1_all, mean_whole_s2_all], color="black", linewidth=2
)
axs[5, 0].plot([x[1] - 0.1, x[1] + 0.1], [mean_ss_s1_all, mean_ss_s2_all], color="black", linewidth=2)
axs[5, 0].plot([x[2] - 0.1, x[2] + 0.1], [mean_ws_s1_all, mean_ws_s2_all], color="black", linewidth=2)

# Plot error bars
axs[5, 0].errorbar(
    x[0] - 0.1, mean_whole_s1_all, yerr=std_whole_s1_all, fmt="", color="black", capsize=5
)
axs[5, 0].errorbar(
    x[0] + 0.1, mean_whole_s2_all, yerr=std_whole_s2_all, fmt="", color="black", capsize=5
)

axs[5, 0].errorbar(x[1] - 0.1, mean_ss_s1_all, yerr=std_ss_s1_all, fmt="", color="black", capsize=5)
axs[5, 0].errorbar(x[1] + 0.1, mean_ss_s2_all, yerr=std_ss_s2_all, fmt="", color="black", capsize=5)

axs[5, 0].errorbar(x[2] - 0.1, mean_ws_s1_all, yerr=std_ws_s1_all, fmt="", color="black", capsize=5)
axs[5, 0].errorbar(x[2] + 0.1, mean_ws_s2_all, yerr=std_ws_s2_all, fmt="", color="black", capsize=5)

# Add significance indicators
y_min = 0.9
bar_height = np.abs(y_min * 0.1)

# For whole
axs[5, 0].plot([x[0] - 0.1, x[0] + 0.1], [y_min + bar_height, y_min + bar_height], 'k-', linewidth=1)
axs[5, 0].text(x[0], y_min + bar_height * 1.2, get_significance(p_val_whole), ha='center', va='bottom')

# For ss
axs[5, 0].plot([x[1] - 0.1, x[1] + 0.1], [y_min + bar_height, y_min + bar_height], 'k-', linewidth=1)
axs[5, 0].text(x[1], y_min + bar_height * 1.2, get_significance(p_val_ss), ha='center', va='bottom')

# For ws
axs[5, 0].plot([x[2] - 0.1, x[2] + 0.1], [y_min + bar_height, y_min + bar_height], 'k-', linewidth=1)
axs[5, 0].text(x[2], y_min + bar_height * 1.2, get_significance(p_val_ws), ha='center', va='bottom')

axs[5, 0].set_xticks(x, ["Whole cells", "SS cells", "WS cells"])
axs[5, 0].set_ylim(0, 1.1)

plt.savefig('./figure/figure_3.svg', bbox_inches='tight', transparent=True)
plt.show()